In [ ]:
from classify import data_pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import pandas as pd

# Rzepa

In [ ]:
#df = data_pipeline("rzepa")
#df.drop(columns=['vectorized'], inplace=True)
#df.to_csv("data_all.csv")

df = pd.read_csv("data_all.csv", index_col = 'id')

In [ ]:
df['Rank'] = None

## Filter I - general good

In [ ]:
def filter_by_conditions(df, condition_list):
    condition_list.append(df['Rank'].isna())
    selection_mask = reduce(lambda x, y: x & y, condition_list)
    df.loc[selection_mask, 'Rank'] = 'Test'
    print(len(df[selection_mask]))
    if len(df[selection_mask]) > 100:
        print('Saving')
        df[selection_mask].sample(100).to_csv("data_test.csv")
    else:
        df[selection_mask].to_csv("data_test.csv")
    return df

def approve(df, name):
    df.loc[df['Rank'] == 'Test', 'Rank'] = name
    return df

def restore(df):
    df.loc[df['Rank'] == 'Test', 'Rank'] = None
    return df

In [ ]:
condition_list = []
condition_list.append(df['ngram_sum_squared_to_total'] > 0.75)
condition_list.append(df['proba'] > 0.5)
condition_list.append(df['weak_count'] < 0.5)

df = filter_by_conditions(df, condition_list)

In [ ]:
# Approved
df = approve(df, "Filter1")

## Filter II - high ngrams

In [ ]:
condition_list = [df['weak_count']  < 0.3,
                  df['ngram_sum_squared_to_total'] > 0.5,
                  df['num_words'] > 2,
                  df['proba'] > 0.5]
df = filter_by_conditions(df, condition_list)

In [ ]:
df = approve(df, 'Filter2')

# Filter III - klimat count

In [ ]:
df['klimat_count'] = df['ngram_sum'] * df['klimat']

In [ ]:
condition_list = [
    df['klimat_count'] > 2,
    df['ngram_sum_squared_to_total'] > 0.33,
]
df = filter_by_conditions(df, condition_list)

In [ ]:
df = approve(df, "Filter3")

# Filter 4

In [ ]:
condition_list = [
    df['klimat_count'] > 0,
    df['proba'] > 0.8,
    df['weak_count'] < 0.5,
    df['num_words'] > 2
    ]
df = filter_by_conditions(df, condition_list)

# Left

In [ ]:
df_left = df[df['Rank'].isna()]
len(df_left)

In [ ]:
df[~df['Rank'].isna()].to_csv('eco_rzepa.csv')